In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import pydicom
import warnings

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt


warnings.filterwarnings("ignore")


DIR_INPUT = '/kaggle/input/vinbigdata-chest-xray-abnormalities-detection'
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

In [ ]:
train_DataFrame = pd.read_csv(f'{DIR_INPUT}/train.csv')


train_DataFrame.fillna(0, inplace=True) # Fill the NAN values with 0

train_DataFrame.loc[train_DataFrame["class_id"] == 14, ['x_max', 'y_max']] = 1.0 # make x_max , y_max of no finding = 1

# FasterRCNN handles class_id==0 as the background.
train_DataFrame["class_id"] = train_DataFrame["class_id"] + 1
train_DataFrame.loc[train_DataFrame["class_id"] == 15, ["class_id"]] = 0

train_DataFrame.shape

In [ ]:
print(train_DataFrame.sort_values(by='image_id').head(50))

num_classes = train_DataFrame["class_id"].nunique()

print('num_classes = ',num_classes)


In [ ]:
imageID = "00150343289f317a0ad5629d5b7d9ef9"

dicom_of_imageID = pydicom.dcmread(f"{DIR_TRAIN}/{imageID}.dicom")


In [ ]:
print(dicom_of_imageID)

In [ ]:
image_from_dicom = dicom_of_imageID.pixel_array * dicom_of_imageID.RescaleSlope + dicom_of_imageID.RescaleIntercept

plt.imshow(image_from_dicom, cmap='gray')

In [ ]:
image_ids = train_DataFrame['image_id'].unique()

validation_percentage = 0.2

len_validation_data = int(validation_percentage*len(image_ids))

valid_ids = image_ids[:len_validation_data]


train_ids = image_ids[len_validation_data:]

print("number of training data = ",len(train_ids))
print("number of validation data = ",len(valid_ids))

In [ ]:
valid_DataFrame = train_DataFrame[train_DataFrame['image_id'].isin(valid_ids)]
train_DataFrame = train_DataFrame[train_DataFrame['image_id'].isin(train_ids)]

In [ ]:
class VinBig_Dataset(Dataset):
    
    def __init__(self, DataFrame, Image_Dir, transforms=None):
        super().__init__()
        self.image_IDs = DataFrame["image_id"].unique()
        self.DF = DataFrame
        self.Image_Dir = Image_Dir
        self.transforms = transforms
        
    def __getitem__(self, index):
        
        image_id = self.image_IDs[index]
        records = self.DF[(self.DF['image_id'] == image_id)]
        records = records.reset_index(drop=True)

        dicom_of_image = pydicom.dcmread(f"{self.Image_Dir}/{image_id}.dicom")
        
        image = dicom_of_image.pixel_array
        
        if "PhotometricInterpretation" in dicom_of_image:
            if dicom_of_image.PhotometricInterpretation == "MONOCHROME1":
                image = np.amax(image) - image
        
        intercept = dicom_of_image.RescaleIntercept if "RescaleIntercept" in dicom_of_image else 0.0
        slope = dicom_of_image.RescaleSlope if "RescaleSlope" in dicom_of_image else 1.0
        
        if slope != 1:
            image = slope * image.astype(np.float64)
            image = image.astype(np.int16)
            
        image += np.int16(intercept)        
        
        image = np.stack([image, image, image])
        image = image.astype('float32')
        image = image - image.min()
        image = image / image.max()
        image = image * 255.0
        image = image.transpose(1,2,0)
       
        if records.loc[0, "class_id"] == 0:
            records = records.loc[[0], :]
        
        boxes = records[['x_min', 'y_min', 'x_max', 'y_max']].values
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)
        labels = torch.tensor(records["class_id"].values, dtype=torch.int64)

        # suppose all instances are not crowd
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([index])
        target['area'] = area
        target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            
            target['boxes'] = torch.tensor(sample['bboxes'])

        if target["boxes"].shape[0] == 0:
            # Albumentation cuts the target (class 14, 1x1px in the corner)
            target["boxes"] = torch.from_numpy(np.array([[0.0, 0.0, 1.0, 1.0]]))
            target["area"] = torch.tensor([1.0], dtype=torch.float32)
            target["labels"] = torch.tensor([0], dtype=torch.int64)
            
        return image, target
    
    def __len__(self):
        return self.image_IDs.shape[0]

In [ ]:
# Albumentations
def get_train_transform():
    return A.Compose([
        A.Flip(0.5),
        A.ShiftScaleRotate(scale_limit=0.1, rotate_limit=45, p=0.25),
        A.LongestMaxSize(max_size=800, p=1.0),

        # FasterRCNN will normalize.
        A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        A.Normalize(mean=(0, 0, 0), std=(1, 1, 1), max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [ ]:
model=torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [ ]:
print(model)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = VinBig_Dataset(train_DataFrame, DIR_TRAIN, get_train_transform())
valid_dataset = VinBig_Dataset(valid_DataFrame, DIR_TRAIN, get_valid_transform())


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
images, targets = next(iter(train_data_loader))

print(targets)

In [ ]:
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [ ]:
boxes = targets[0]['boxes'].cpu().numpy().astype(np.int32)
sample = images[0].permute(1,2,0).cpu().numpy()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

for box in boxes:
    cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2], box[3]),
                  (220, 0, 0), 3)
    
ax.set_axis_off()
ax.imshow(sample)

In [ ]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0

In [ ]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

num_epochs = 30

In [ ]:
loss_hist = Averager()
itr = 1

for epoch in range(num_epochs):
    loss_hist.reset()
    
    for images, targets in train_data_loader:
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        
        loss_hist.send(loss_value)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if itr % 100 == 0:
            print(f"Iteration #{itr} loss: {loss_hist.value}")

        itr += 1
        
        # !!!REMOVE THIS!!!
        break
    
    # update the learning rate
    if lr_scheduler is not None:
        lr_scheduler.step()

    print(f"Epoch #{epoch} loss: {loss_hist.value}")   
    # print("Saving epoch's state...")
    # torch.save(model.state_dict(), f"model_state_epoch_{epoch}.pth")

In [ ]:
torch.save(model.state_dict(), "model.pth")
from IPython.display import FileLink
FileLink(r'model.pth')